In [1]:
from datasets import CXRDataset
import numpy as np

In [2]:
IMG_DIR = './CXR8/Images'
TRAIN_LIST_FILE = './CXR8/Labels/train_list.txt'
VAL_LIST_FILE = './CXR8/Labels/val_list.txt'
TEST_LIST_FILE = './CXR8/Labels/test_list.txt'
AUX_LIST_FILE = './CXR8/Labels/aux_list.txt'

In [3]:
CLASS_NAMES = [
    'Atelectasis', 'Cardiomegaly', 'Effusion',
    'Infiltration', 'Mass', 'Nodule', 'Pneumonia',
    'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema',
    'Fibrosis', 'Pleural_Thickening', 'Hernia'
]

labels = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
]

index = 2
dis = np.array(list(map(CLASS_NAMES.__getitem__, np.nonzero(labels[index])[0])))

print(dis if len(dis) != 0 else ['Normal'])

['Atelectasis' 'Cardiomegaly']


In [5]:
val = CXRDataset(IMG_DIR, AUX_LIST_FILE)
val.load()

print(len(val))
val.print_img(0)
val.print_img(1)
val.print_img(2)

load took 0.09551763534545898[s] to execute.

5
print
[[0.99607843 0.99607843 1.         ... 0.9882353  1.         0.65882355]
 [1.         0.99607843 0.99215686 ... 0.99215686 1.         0.65882355]
 [1.         0.99607843 0.99215686 ... 1.         1.         0.65882355]
 ...
 [0.09411765 0.09411765 0.09411765 ... 0.5882353  0.6862745  0.48235294]
 [0.09411765 0.09411765 0.09411765 ... 0.58431375 0.68235296 0.47058824]
 [0.09411765 0.09411765 0.09411765 ... 0.5803922  0.68235296 0.47058824]]
print
[[0.34509805 0.29411766 0.26666668 ... 0.5803922  0.5764706  0.5882353 ]
 [0.30980393 0.25882354 0.23137255 ... 0.49411765 0.49411765 0.50980395]
 [0.2784314  0.23921569 0.2        ... 0.40392157 0.4117647  0.42352942]
 ...
 [0.16470589 0.17254902 0.16862746 ... 0.16862746 0.15294118 0.14117648]
 [0.16078432 0.16862746 0.1764706  ... 0.16470589 0.14901961 0.13725491]
 [0.16078432 0.16862746 0.18431373 ... 0.16078432 0.14509805 0.13725491]]
print
[[0.6156863  0.5921569  0.5882353  ... 0.     